## <font>Análise de Dados de Sensores IoT em Tempo Real com Apache Spark Streaming e Apache Kafka</font>

![title](imagens/kafka.png)

In [1]:
# Versão da Linguagem Python
from platform import python_version
print('Versão da Linguagem Python Usada Neste Jupyter Notebook:', python_version())

Versão da Linguagem Python Usada Neste Jupyter Notebook: 3.9.13


## Definição do problema e fonte de dados

Neste projeto vamos implementar uma solução completa de análise de dados em tempo real usando o Apache Spark e o Apache Kafka. Vamos coletar dados de sensores IoT (Internet of Things) e, em tempo real (à medida que os dados são gerados e coletados), realizar o trabalho de análise e entregar a resposta para um determinado problema de negócio.


Este projeto é composto de diversas partes uma vez que toda a infraestrutura necessária será construída passo a passo. 


## Coletando os dados dos sensores IoT

Para coletar dados dos sensores IoT (Internet of Things), vamos seguir as seguintes etapas:

1-	Abrir o terminal ou prompt de comando e acessar a pasta com o simulador.

2-	Execute o comando abaixo para gerar um arquivo com 10.000 leituras de sensores IoT (pode gerar quantos registros você desejar).

>python simulador.py 1000 > ../dados/dados_sensores.txt

## Instalação do Apache Kafka

> Todos os comandos para instalação e execução do Apache Kafka no Windows estão documentados no README deste projeto.

In [2]:
# Para atualizar um pacote, execute o comando abaixo no terminal ou prompt de comando:
# pip install -U nome_pacote

# Para instalar a versão exata de um pacote, execute o comando abaixo no terminal ou prompt de comando:
#!pip install nome_pacote==versão_desejada

# Depois de instalar ou atualizar o pacote, reinicie o jupyter notebook.

# Instala o pacote watermark. 
# Esse pacote é usado para gravar as versões de outros pacotes usados neste jupyter notebook.
#!pip install -q -U watermark

In [3]:
#!pip install -q findspark

In [4]:
# Importa o findspark e inicializa
import findspark
findspark.init()

In [5]:
# Import required modules
import pyspark
from pyspark.streaming import StreamingContext
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, DoubleType
from pyspark.sql.functions import col, from_json

> Precisamos incluir o conector de integração do Spark Streaming com o Apache Kafka. Fique atento à versão do PySpark que está sendo usada.

https://spark.apache.org/docs/latest/structured-streaming-kafka-integration.html

In [9]:
# Conector
import os
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-sql-kafka-0-10_2.13:3.5.0 pyspark-shell'

In [10]:
# Versões dos pacotes usados neste jupyter notebook
%reload_ext watermark
%watermark -a "Data Science Academy" --iversions

Author: Data Science Academy

pyspark  : 3.5.0
findspark: 2.0.1



## Criando a Sessão Spark

In [1]:
# Cria a sessão Spark
spark = SparkSession.builder.appName("Mini-Projeto6").getOrCreate()

NameError: name 'SparkSession' is not defined

## Leitura do Kafka Spark Structured Stream

In [ ]:
# Vamos criar uma subscrição no tópico que tem o streaming de dados que desejamos "puxar" os dados.
df = spark \
  .readStream \
  .format("kafka") \
  .option("kafka.bootstrap.servers", "localhost:9092") \
  .option("subscribe", "dsamp6") \
  .load()

## Definição do Schema da Fonte de Dados

In [ ]:
# Definimos o schema dos dados que desejamos capturar para análise (temperatura)
esquema_dados_temp = StructType([StructField("leitura", 
                                             StructType([StructField("temperatura", DoubleType(), True)]), True)])

In [ ]:
# Definimos o schema global dos dados no streaming
esquema_dados = StructType([ 
    StructField("id_sensor", StringType(), True), 
    StructField("id_equipamento", StringType(), True), 
    StructField("sensor", StringType(), True), 
    StructField("data_evento", StringType(), True), 
    StructField("padrao", esquema_dados_temp, True)
])

## Parse da Fonte de Dados

In [ ]:
# Capturamos cada linha de dado (cada valor) como string
df_conversao = df.selectExpr("CAST(value AS STRING)")

In [ ]:
# Parse do formato JSON em dataframe
df_conversao = df_conversao.withColumn("jsonData", from_json(col("value"), esquema_dados)).select("jsonData.*")

In [ ]:
df_conversao.printSchema()

## Preparamos o Dataframe 

Esse dataframe está no formato que precisamos para análise.

In [ ]:
# Renomeamos as colunas para simplificar nossa análise
df_conversao_temp_sensor = df_conversao.select(col("padrao.leitura.temperatura").alias("temperatura"), 
                                               col("sensor"))

In [ ]:
df_conversao_temp_sensor.printSchema()

In [ ]:
# Não podemos visualizar o dataframe, pois a fonte é de streaming
# df_conversao_temp_sensor.head()

## Análise de Dados em Tempo Real

In [ ]:
# Aqui temos o objeto que irá conter nossa análise, o cálculo da média das temperaturas por sensor
df_media_temp_sensor = df_conversao_temp_sensor.groupby("sensor").mean("temperatura")

In [ ]:
df_media_temp_sensor.printSchema()

In [ ]:
# Renomeamos as colunas para simplificar nossa análise
df_media_temp_sensor = df_media_temp_sensor.select(col("sensor").alias("sensor"), 
                                                   col("avg(temperatura)").alias("media_temp"))

In [ ]:
df_media_temp_sensor.printSchema()

Abaixo abrimos o streaming para análise de dados em tempo real, imprimindo o resultado no console.

In [ ]:
# Objeto que inicia a consulta ao streaming com formato de console
query = df_media_temp_sensor.writeStream.outputMode("complete").format("console").start()

Envie novos arquivos para o Kafka a fim de ver a análise em tempo real por aqui. Clique no botão Stop no menu superior para interromper a célula a qualquer momento.

In [ ]:
# Executamos a query do streaming e evitamos que o processo seja encerrado
query.awaitTermination()

In [ ]:
query.status

In [ ]:
query.lastProgress

In [ ]:
query.explain()

## Análise de Dados em Tempo Real

In [ ]:
# Objeto que inicia a consulta ao streaming com formato de memória (cria tabela temporária)
query_memoria = df_media_temp_sensor \
    .writeStream \
    .queryName("dsa") \
    .outputMode("complete") \
    .format("memory") \
    .start()

In [ ]:
# Streams ativados
spark.streams.active

In [ ]:
# Vamos manter a query executando por algum tempo e aplicando SQL aos dados em tempo real
from time import sleep

for x in range(10):
    
    spark.sql("select sensor, round(media_temp, 2) as media from dsa where media_temp > 65").show()
    sleep(3)
    
query_memoria.stop()

# Fim